## Create SpotMAX model description and package it

### Links

[Create model from scratch](https://github.com/bioimage-io/core-bioimage-io-python/blob/main/presentations/create_ambitious_sloth.ipynb)

[Inspect and package model](https://github.com/bioimage-io/spec-bioimage-io/blob/main/example/load_model_and_create_your_own.ipynb)

[Additional discussion](https://github.com/bioimage-io/collection/issues/104)

[Model usage](https://github.com/bioimage-io/core-bioimage-io-python/blob/main/example/model_usage.ipynb)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path

from bioimageio.spec.model.v0_5 import (
    Author,
    AxisId,
    BatchAxis,
    ChannelAxis,
    CiteEntry,
    EnvironmentFileDescr,
    DatasetId,
    Doi,
    FileDescr,
    HttpUrl,
    Identifier,
    InputTensorDescr,
    IntervalOrRatioDataDescr,
    LicenseId,
    LinkedDataset,
    ModelDescr,
    OrcidId,
    ParameterizedSize,
    PytorchStateDictWeightsDescr,
    ScaleRangeDescr,
    ScaleRangeKwargs,
    SpaceInputAxis,
    TensorId,
    TorchscriptWeightsDescr,
    WeightsDescr,
)
from bioimageio.spec.pretty_validation_errors import (
    enable_pretty_validation_errors_in_ipynb,
)

from spotmax import spotmax_path

### Model Input

In [4]:
import numpy as np

model_folder_root = Path(spotmax_path) / 'BioImageIO' / 'SpotMAX_UNet_2D'
input_sample_path = model_folder_root / 'input_sample.npy'
input_sample = np.load(str(input_sample_path))
Z, Y, X = input_sample.shape[-3:]
Z, Y, X, input_sample.dtype


(43, 123, 167, dtype('uint8'))

In [4]:
input_descr = InputTensorDescr(
    id=TensorId('mNeon'),
    axes=[
        BatchAxis(),
        SpaceInputAxis(
            id=AxisId('z'),
            size=43,
            concatenable=False
        ),
        SpaceInputAxis(
            id=AxisId('y'),
            size=123,
            concatenable=False
        ),
        SpaceInputAxis(
            id=AxisId('x'),
            size=167,
            concatenable=False),
    ],
    test_tensor=FileDescr(source=input_sample_path),
    # sample_tensor=FileDescr(source=input_sample_path),
    data=IntervalOrRatioDataDescr(type='uint8'),
)

computing SHA256 of input_sample.npy (result: b3ebf3e5ef1f0547c2bfc70d2a4e80d23a2d3676c984752d49cdbe3d2525f161): 100%|██████████| 883391/883391 [00:00<00:00, 177113308.07it/s]


### Model Output

In [6]:
output_sample_path = model_folder_root / 'output_sample_mask.npy'
output_sample = np.load(str(output_sample_path))
Z, Y, X = output_sample.shape[-3:]
Z, Y, X, output_sample.dtype

(43, 123, 167, dtype('uint8'))

In [7]:
from bioimageio.spec.model.v0_5 import (
    OutputTensorDescr, SizeReference, SpaceOutputAxis
)

output_descr = OutputTensorDescr(
    id=TensorId('mask'),
    description='predicted boolean mask of spot areas',
    axes=[
        BatchAxis(),
        # ChannelAxis(channel_names=[Identifier('prediction')]),
        SpaceOutputAxis(id=AxisId('z'), size=43),
        SpaceOutputAxis(id=AxisId('y'), size=123),
        SpaceOutputAxis(id=AxisId('x'), size=167),
    ],
    test_tensor=FileDescr(source=output_sample_path),
    data=IntervalOrRatioDataDescr(type='uint8'),
)

computing SHA256 of output_sample_mask.npy (result: 41ac73f1ca089e6fe631b825bf6e3ab70c0d2d3cf59a24a494ad293fc8541801): 100%|██████████| 883391/883391 [00:00<00:00, 292647532.17it/s]


### Model Architecture

In [7]:
import torch

from bioimageio.spec.model.v0_5 import (
    ArchitectureFromFileDescr,
    Version,
)

model_py_path = Path(spotmax_path) / 'nnet' / 'model.py'

pytorch_version = Version(torch.__version__)

pytorch_architecture = ArchitectureFromFileDescr(
    source=model_py_path,
    callable=Identifier('Model'),
    kwargs=dict(
        model_type='2D', 
        preprocess_across_experiment=False,
        preprocess_across_timepoints=False,
        gaussian_filter_sigma=0,
        remove_hot_pixels=True,
        config_yaml_filepath='spotmax/nnet/config.yaml', 
        PhysicalSizeX=0.06725,
        resolution_multiplier_yx=1, 
        use_gpu=True, 
        save_prediction_map=False, 
        verbose=True,
    )
)

computing SHA256 of model.py (result: 3d01aba04140ad545015cafab39ec5cd721b6a937ca983dcb3ee8dd1776f0f40): 100%|██████████| 16221/16221 [00:00<?, ?it/s]


### Create model description

In [8]:
model_descr = ModelDescr(
  name='SpotMAX-AI',
  description=(
    'U-Net 2D trained on images containing diffraction-limited fluorescent '
    'spots. The model is trained to return a boolean mask of spot areas.'
  ),
  covers=[model_folder_root / 'cover.png'],
  authors=[
      Author(
        name='Francesco Padovani',
        affiliation='Helmholtz Munich',
        email='padovaf@tcd.ie',
        github_user='ElpadoCan',
        orcid=OrcidId('0000-0003-2540-8240')
      )
  ],
  cite=[
    CiteEntry(
      text=(
        'Padovani, F., Čavka, I., Neves, A. R. R., López, C. P., Al-Refaie, N., '
        'Bolcato, L., Chatzitheodoridou, D., Chadha, Y., Su, X.A., Lengefeld, J., '
        'Cabianca D. S., Köhler, S., Schmoller, K. M. SpotMAX: a generalist '
        'framework for multi-dimensional automatic spot detection and quantification, '
        'bioRxiv (2024) DOI: 10.1101/2024.10.22.619610'
      ),
      doi=Doi('10.1101/2024.10.22.619610'),
    )
  ],
  license=LicenseId('GPL-3.0-only'),
  documentation=model_folder_root / 'README.md',
  git_repo=HttpUrl('https://github.com/ElpadoCan/SpotMAX'),
  tags= [
      'spot-detection',
      'diffraction-limited-spots',
      'pytorch',
      'fluorescence-light-microscopy',
      'spotmax',
  ],
  # training_data=LinkedDataset(id=DatasetId('uplifting-ice-cream')),
  inputs=[input_descr],
  outputs=[output_descr],
  weights=WeightsDescr(
      pytorch_state_dict=PytorchStateDictWeightsDescr(
          source=model_folder_root / 'unet_best.pth',
          architecture=pytorch_architecture,
          pytorch_version=pytorch_version,
          dependencies=EnvironmentFileDescr()
      ),
  ),
  attachments=[FileDescr(source=model_folder_root / 'model_usage.py')],
)

computing SHA256 of unet_best.pth (result: c1e276f8aa450bac8ecb3b04068754182de77cf20aefe1e4d01729f71804a901): 100%|██████████| 69122085/69122085 [00:00<00:00, 263527784.13it/s]
computing SHA256 of model_usage.py (result: 3fbae1fcd7e507f94ff9bfc1cdcb800df61fddd3b4953a7f441f3e347c9efe8c): 100%|██████████| 1637/1637 [00:00<00:00, 820711.89it/s]
2024-11-29 20:51:04.582 | Level 30 | bioimageio.spec.model.v0_5:_validate_documentation:2106 - documentation: No '# Validation' (sub)section found in D:\OneDrive\01_Postdoc_HMGU\GitHub\spotMAX_v2\spotmax\BioImageIO\SpotMAX_UNet_2D\README.md.


### Test the model

In [9]:
from bioimageio.core import test_model

validation_summary = test_model(model_descr)

2024-11-29 20:51:21.699 | DEBUG    | bioimageio.core._resource_tests:enable_determinism:93 - module 'tensorflow._api.v2.random' has no attribute 'seed'
2024-11-29 20:51:21.700 | INFO     | bioimageio.core._resource_tests:_test_model_inference:226 - starting 'Reproduce test outputs from test inputs (pytorch_state_dict)'
2024-11-29 20:51:21.706 | WARNING  | bioimageio.spec.get_conda_env:_get_default_pytorch_env:142 - UPDATE NEEDED: Specify pins for additional pytorch==2.3.1+cpu dependencies!
2024-11-29 20:51:28.202 | INFO     | bioimageio.core._resource_tests:_test_model_inference_parametrized:317 - Testing inference with 2 different input tensor sizes


In [10]:
validation_summary.display()


|        ❌        | bioimageio format validation failed |
|       ---       |                 ---                 |
| source          | in-memory                           |
| format version  | model 0.5.3                         |
| bioimageio.spec | 0.5.3.5                             |
| bioimageio.core | 0.7.0                               |

|  ❓  |           location           |                                                                                                                                                                                                                                                                                                                                              detail                                                                                                                                                                                                                                                                                                                                              |
| --- |             ---              |                                                                                                                                                                                                                                                                                                                                               ---                                                                                                                                                                                                                                                                                                                                                |
| ✔️  | `.`                          | initialized ModelDescr to describe model 0.5.3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| ✔️  | `type`                       | Has expected resource type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| ❌   | `weights.pytorch_state_dict` | Reproduce test outputs from test inputs (pytorch_state_dict)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 🐍   | `weights.pytorch_state_dict` | recommended conda env (Reproduce test outputs from test inputs (pytorch_state_dict))<br><pre><code>channels: [pytorch, conda-forge, nodefaults]</code><br><code>dependencies: [cpuonly, 'numpy >=2,<3', pip, pytorch==2.3.1+cpu, torchaudio, torchvision]</code><br><code></code></pre>                                                                                                                                                                                                                                                                                                                                                                                                          |
| 🐍   | `weights.pytorch_state_dict` | conda compare ({d.name}):<br>cpuonly not found<br>numpy found but mismatch. Specification pkg: numpy[version='>=2,<3'], Running pkg: numpy==1.26.4=py310hf667824_0<br>pytorch not found<br>torchaudio not found<br>                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| ❌   | `weights.pytorch_state_dict` | The 'pytorch_state_dict' model adapter could not be created in this environment:<br>ImportError(attempted relative import with no known parent package).<br>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|     |                              | [_resource_tests.py](_resource_tests.py "c:/Users/elpad/miniforge3/envs/acdc/lib/site-packages/bioimageio/core/_resource_tests.py"), line 234, in _test_model_inference<br>`    with create_prediction_pipeline(`<br>[_prediction_pipeline.py](_prediction_pipeline.py "c:/Users/elpad/miniforge3/envs/acdc/lib/site-packages/bioimageio/core/_prediction_pipeline.py"), line 351, in create_prediction_pipeline<br>`    model_adapter = model_adapter or create_model_adapter(`<br>[_model_adapter.py](_model_adapter.py "c:/Users/elpad/miniforge3/envs/acdc/lib/site-packages/bioimageio/core/model_adapters/_model_adapter.py"), line 137, in create<br>`    raise ValueError(`              |
| ❌   | `weights.pytorch_state_dict` | Run pytorch_state_dict inference for parametrized inputs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| ❌   | `weights.pytorch_state_dict` | The 'pytorch_state_dict' model adapter could not be created in this environment:<br>ImportError(attempted relative import with no known parent package).<br>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|     |                              | [_resource_tests.py](_resource_tests.py "c:/Users/elpad/miniforge3/envs/acdc/lib/site-packages/bioimageio/core/_resource_tests.py"), line 371, in _test_model_inference_parametrized<br>`    with create_prediction_pipeline(`<br>[_prediction_pipeline.py](_prediction_pipeline.py "c:/Users/elpad/miniforge3/envs/acdc/lib/site-packages/bioimageio/core/_prediction_pipeline.py"), line 351, in create_prediction_pipeline<br>`    model_adapter = model_adapter or create_model_adapter(`<br>[_model_adapter.py](_model_adapter.py "c:/Users/elpad/miniforge3/envs/acdc/lib/site-packages/bioimageio/core/model_adapters/_model_adapter.py"), line 137, in create<br>`    raise ValueError(` |


### Package the model

Save all the model files to a zip file that can be uploaded to BioImage.IO

In [30]:
from pathlib import Path

from bioimageio.spec import save_bioimageio_package

print(
    'Package path:',
    save_bioimageio_package(
        model_descr, 
        output_path=model_folder_root / 'SpotMAX_AI_2D.zip'
    ),
)

computing SHA256 of model_usage.py (result: 3fbae1fcd7e507f94ff9bfc1cdcb800df61fddd3b4953a7f441f3e347c9efe8c): 100%|██████████| 1/1 [00:00<00:00, 1046.22it/s]
computing SHA256 of input_sample.npy (result: b3ebf3e5ef1f0547c2bfc70d2a4e80d23a2d3676c984752d49cdbe3d2525f161): 100%|██████████| 7/7 [00:00<00:00, 1754.73it/s]
computing SHA256 of output_sample_mask.npy (result: 41ac73f1ca089e6fe631b825bf6e3ab70c0d2d3cf59a24a494ad293fc8541801): 100%|██████████| 7/7 [00:00<00:00, 2340.94it/s]
computing SHA256 of model.py (result: 3d01aba04140ad545015cafab39ec5cd721b6a937ca983dcb3ee8dd1776f0f40): 100%|██████████| 1/1 [00:00<00:00, 1003.90it/s]
computing SHA256 of unet_best.pth (result: c1e276f8aa450bac8ecb3b04068754182de77cf20aefe1e4d01729f71804a901): 100%|██████████| 528/528 [00:00<00:00, 3460.21it/s]


ValueError: Exported package 'SpotMAX_AI_2D.zip' is invalid: ValidationSummary:

|        ❌        |                                        bioimageio validation failed                                        |
|       ---       |                                                    ---                                                     |
| source          | d:\OneDrive\01_Postdoc_HMGU\GitHub\spotMAX_v2\spotmax\BioImageIO\SpotMAX_AI_2D.zip.unzip\SpotMAX_AI_2D.zip |
| format version  | model 0.5.3                                                                                                |
| bioimageio.spec | 0.5.3.3                                                                                                    |

|  ❓  |                  location                  |                                                                                                                               detail                                                                                        